# 고객요청의도 분석하기

In [ ]:
!pip install langchain openai chromadb tiktoken pypdf unstructured sentence-transformers langchain-community

# 데이터 읽어오기

In [3]:
import pandas as pd

In [4]:
json_file_path = '민원(콜센터) 질의응답_금융보험_잔고 및 거래내역_Training.json'
df = pd.read_json(json_file_path)
df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,금융/보험,잔고 및 거래내역,A4091,고객,1,인증서,,Q,집과 직장에서 인터넷 뱅킹을 이용가능하나요?,,,,"가정, 직장, 인터넷 뱅킹",인터넷뱅킹/금융서비스,"직장,금융서비스"
1,금융/보험,잔고 및 거래내역,A4091,상담사,2,,인증서,Q,,공인인증서가 있습니까?,,,공인인증서,공인인증서/공인인증,"공인인증서,공인인증"
2,금융/보험,잔고 및 거래내역,A4091,고객,3,인증서,,A,,,네,,,,
3,금융/보험,잔고 및 거래내역,A4091,상담사,4,,인증서,A,,,,가능합니다. 공인인증서를 복사해서 사용할 수 있습니다.,"공인인증서, 복사",공인인증서/공인인증,"복사,공인인증"
4,금융/보험,잔고 및 거래내역,A4091,고객,5,인증서,,Q,인증서 복사는 어떻게 하죠?,,,,"복사, 방법",,방법
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74353,금융/보험,잔고 및 거래내역,A32059,고객,16,거래내역조회,,Q,12명 총 입금액이 얼마인가요?,,,,"입금액,얼마","입금액,얼마,입금된금액","얼마,입금된금액"
74354,금융/보험,잔고 및 거래내역,A32059,상담사,17,,거래내역조회,A,,,,350만원 이십니다.,,,
74355,금융/보험,잔고 및 거래내역,A32059,고객,18,거래내역조회,,A,,,잘 알겠습니다.,,,,
74356,금융/보험,잔고 및 거래내역,A32059,상담사,19,,거래내역조회,Q,,더 필요하신 내용 있으신가요?,,,,,


In [5]:
len(df)

74358

In [6]:
df.columns

Index(['도메인', '카테고리', '대화셋일련번호', '화자', '문장번호', '고객의도', '상담사의도', 'QA',
       '고객질문(요청)', '상담사질문(요청)', '고객답변', '상담사답변', '개체명 ', '용어사전', '지식베이스'],
      dtype='object')

# 각 column별 unique한 값 확인하기(생략)

# 고객의도가 '인증서'와 관련된 데이터만 필터링하기

In [ ]:
고객의도_리스트 = df['고객의도'].unique()
for idx, 고객의도 in enumerate(고객의도_리스트):
    print(f'{idx}: {고객의도}')

In [9]:
인증서_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '인증서' in 의도]
인증서_키워드_포함_고객의도_리스트

['인증서',
 '공인인증서발급문의',
 '공인인증서갱신',
 '카카오페이 인증서',
 '공인인증서암호',
 '인증서 연장',
 '인증서복사',
 '공인인증서',
 '공인인증서 연장',
 '공인인증서문의',
 '공인인증서오류',
 '공인인증서복사',
 '공인인증서만료',
 '기업공인인증서',
 '공인인증서발급',
 '공인인증서관리',
 '공인인증서 문의',
 '공인인증서 갱신',
 '타행인증서 등록',
 '공인인증서 연장신청',
 '모바일 인증서관리',
 '인증서 등록']

In [10]:
filtered_df = df[(df['고객의도'] == '인증서') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
0,금융/보험,잔고 및 거래내역,A4091,고객,1,인증서,,Q,집과 직장에서 인터넷 뱅킹을 이용가능하나요?,,,,"가정, 직장, 인터넷 뱅킹",인터넷뱅킹/금융서비스,"직장,금융서비스"
40,금융/보험,잔고 및 거래내역,A4119,고객,1,인증서,,Q,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...,,,,"스마트뱅킹,공인인증서,비밀번호,초기화면",스마트뱅킹/공인인증서/비밀번호/입력/초기화면,"공인인증서,초기화면"
11343,금융/보험,잔고 및 거래내역,A6986,고객,1,인증서,,Q,인증서 연장 신청 하려고 하는데요?,,,,인증서.연장.신청,연장/갱신.신청/청구,"인증서.연장.신청,청구"
20872,금융/보험,잔고 및 거래내역,A12471,고객,1,인증서,,Q,인증서 연장을 어떻게 하죠?,,,,연장,연장/갱신,"연장,갱신"
21034,금융/보험,잔고 및 거래내역,A12520,고객,1,인증서,,Q,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...,,,,스마트뱅킹/공인인증서/비밀번호/초기화면,스마트뱅킹/공인인증서/비밀번호/입력/초기화면,"공인인증서,초기화면"
39228,금융/보험,잔고 및 거래내역,A17086,고객,1,인증서,,Q,인증서 파일로 첨부할수있나요,,,,"인증서, e-mail","인증서/공인인증, e-mail/컴퓨터통신망","e-mail,컴퓨터통신망"
62036,금융/보험,잔고 및 거래내역,A29889,고객,1,인증서,,Q,스마트뱅킹을 하는데 공인인증서 비밀번호를 입력했는데 자꾸 초기 화면으로 돌아가요. ...,,,,스마트뱅킹/공인인증서/비밀번호/초기화면,"스마트뱅킹/모바일뱅킹, 비밀번호/암호","공인인증서,암호"


In [11]:
filtered_df = df[(df['고객의도'] == '공인인증서발급문의') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
5992,금융/보험,잔고 및 거래내역,A6336,고객,1,공인인증서발급문의,,Q,"공인인증서를 발급받으려고하는데요,은행에 가지않고 받을수있는 방법이 있나요?",,,,"공인인증서, 발급, 은행, 방법","공인인증서/전자증명서, 발급/개설, 은행/금융기관, 방법/방식","발급,방식"


# 이하 생략

# 고객의도가 '공인인증서'와 관련된 데이터만 필터링하기

In [12]:
공인인증서_키워드_포함_고객의도_리스트 = [의도 for 의도 in 고객의도_리스트 if '고객의도_리스트' if '공인인증서' in 의도]
공인인증서_키워드_포함_고객의도_리스트

['공인인증서발급문의',
 '공인인증서갱신',
 '공인인증서암호',
 '공인인증서',
 '공인인증서 연장',
 '공인인증서문의',
 '공인인증서오류',
 '공인인증서복사',
 '공인인증서만료',
 '기업공인인증서',
 '공인인증서발급',
 '공인인증서관리',
 '공인인증서 문의',
 '공인인증서 갱신',
 '공인인증서 연장신청']

In [13]:
filtered_df = df[(df['고객의도'] == '공인인증서') & (df['문장번호'] == 1)]
filtered_df

,도메인,카테고리,대화셋일련번호,화자,문장번호,고객의도,상담사의도,QA,고객질문(요청),상담사질문(요청),고객답변,상담사답변,개체명,용어사전,지식베이스
12302,금융/보험,잔고 및 거래내역,A7095,고객,1,공인인증서,,Q,공인인증서 유효기간이 끝났다고 하는데 어떻게 하죠?,,,,"공인인증서,유효기간",공인인증서/유효기간,유효기간
27276,금융/보험,잔고 및 거래내역,A13170,고객,1,공인인증서,,Q,공인인증서 관련 질문드리려고 합니다.,,,,"공인인증서, 질문","공인인증서/인증서, 질문/문의","질문,문의"
61670,금융/보험,잔고 및 거래내역,A29755,고객,1,공인인증서,,Q,보험료 납입내역을 좀 확인하려고 인터넷에 접속했는데 공인인증서 비밀번호가 틀렸다고 ...,,,,인터넷/접속/공인인증서/비밀번호/보험료/납입내역/확인,"인터넷/온라인, 비밀번호/암호, 보험료/금액, 납입내역/납부내역, 확인/체크","접속,체크"


# 사용자 질문으로부터 '고객요청의도' 추출하기

# OpenAI API Key 설정

In [31]:
OPENAI_KEY = "your_OPENAI_key"

In [32]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_KEY)

# create_tagging_chain으로 고객 요청의도 추출하기

In [33]:
from langchain.chains import create_tagging_chain

# 고객요청의도 추출을 위한 schema 설정
schema = {
    "properties": {
        "고객요청의도": {
            "type": "string",
            "enum": ['인증서',
                     '공인인증서'
            ]
        }
    }
}
chain = create_tagging_chain(schema, llm)

In [34]:
chain_run_result = chain.run("공인인증서 유효기간이 끝났다고 하는데 어떻게 하죠?	")
chain_run_result

{'고객요청의도': '공인인증서'}

In [35]:
chain_run_result = chain.run("집과 직장에서 인터넷 뱅킹을 이용가능하나요?	")
chain_run_result

{'고객요청의도': '인증서'}

In [36]:
chain_run_result = chain.run("안녕하세요 A은행이죠?")
chain_run_result

{'고객요청의도': None}

# 위와 같이 질문 유형에 따라 공인인증서와 인증서를 구분 할 수 있다